In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Data Augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

# Load only the "on-road" dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    "D:/Salman_Projects/Municipality/municipality/train",
    labels="inferred",
    label_mode="int",  # Only one label class: "on-road"
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

# Define the model using MobileNetV2 as a base
base_model = keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(img_height, img_width, 3),
    include_top=False,
)

base_model.trainable = False

# Build the model
inputs = keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)  # Predicting 0 or 1
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

# Train the model with only "on-road" data
epochs = 10
history = model.fit(train_ds, epochs=epochs)

# Print training history instead of plotting
print("\nTraining history:")
for epoch in range(epochs):
    acc = history.history["accuracy"][epoch]
    loss = history.history["loss"][epoch]
    print(f"Epoch {epoch + 1}/{epochs} - Accuracy: {acc:.4f}, Loss: {loss:.4f}")

# Save the modelc
model.save("on_road_anomaly_detector.keras")


Found 2704 files belonging to 2 classes.
Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 61s 591ms/step - accuracy: 0.9835 - loss: 0.0548
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 51s 595ms/step - accuracy: 1.0000 - loss: 4.3096e-04
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 49s 579ms/step - accuracy: 1.0000 - loss: 3.0161e-04
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 51s 597ms/step - accuracy: 1.0000 - loss: 2.6366e-04
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 50s 585ms/step - accuracy: 1.0000 - loss: 1.8731e-04
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 50s 582ms/step - accuracy: 1.0000 - loss: 1.3680e-04
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 51s 601ms/step - accuracy: 1.0000 - loss: 1.0042e-04
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 50s 586ms/step - accuracy: 1.0000 - loss: 8.8397e-05
Epoch 9/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 50s 593ms/step - accuracy: 1.0000 - loss: 7.7801e-05
Epoch 10/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 50s 587ms/step - accuracy: 1.0000 - loss: 7.2476e-05

Training history:
Epoch 1/10 - Accuracy: 0.9967, Lo

In [49]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = keras.models.load_model("on_road_anomaly_detector.keras")

# Function to load and preprocess the image
def preprocess_image(img_path):
    img_height, img_width = 224, 224  # Dimensions used for training
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = keras.applications.mobilenet_v2.preprocess_input(img_array)  # Preprocess as per MobileNetV2
    return img_array

# Predict if the car is on-road or off-road
def predict_car_image(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    
    # Assuming your model output is a single value representing the probability
    if prediction[0] > 0.5:
        print("Prediction: The car is on the road.")
    else:
        print("Prediction: The car is off-road.")

# Example usage
img_path = "D:/Salman_Projects/Municipality/municipality/carimage.jpeg"  # Replace with the path to your test car image
predict_car_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: The car is off-road.
